In [5]:
# import a bounch of stuff that allow us to play with sqlalchemy as ORM
from sqlalchemy import create_engine
from sqlalchemy import Column, DateTime, String, Text, Integer, ForeignKey, func, Boolean, update, or_, desc # some datatype and filters
from sqlalchemy.orm import relationship, backref, sessionmaker,session # session plus a little more
from sqlalchemy.ext.declarative import declarative_base

# Create SQLite DB with a Table

In [16]:
# we inicialize the declarative mode 
Base = declarative_base()

# we create a class that will map the columns in the db
class MyTable(Base):
    # table name
    __tablename__ = 'mytable'
    # all columns 
    id = Column('id',Integer, primary_key=True, autoincrement=True )
    address = Column('address',Text )
    price = Column('price',Text)
    rooms = Column('rooms',Text)
    bathrooms = Column('bathrooms',Text)
    area = Column('area',Text)
    url = Column('url',Text) # Text, can be also Integer, Boolean 

# were we gonna store the SQLite DB
db = create_engine("sqlite:///database.sqlite")
# it will plot the log of the dialog of the db
db.echo = False  # Try changing this to True 

# here we create the table if not present
session = sessionmaker()
session.configure(bind=db)
Base.metadata.create_all(db)

# we need to populate the table, we will use the data.csv as example

In [17]:
import csv
# like in the first file, we loop thru rows 
file = open('data.csv','r')
reader = csv.reader(file)
for row in reader:
    # one by one, and we print the results
    print(row)
    
file.close()

['name', 'price', 'rooms', 'bathrooms', 'area', 'url']
['Trilocale via pinerolo, Rivalta di Torino', '€ 78.000', '3', '1', 'None', 'https://www.immobiliare.it/63854720-Vendita-Trilocale-via-pinerolo-Rivalta-di-Torino.html']
['Bilocale via Luigi Salvatore Cherubini 20, Torino', '€ 49.000', '2', '1', '55', 'https://www.immobiliare.it/63858966-Vendita-Bilocale-via-Luigi-Salvatore-Torino.html']
['Quadrilocale via Luigi Salvatore Cherubini 20, Torino', '€ 79.000', '4', '1', '85', 'https://www.immobiliare.it/63857262-Vendita-Quadrilocale-via-Luigi-Salvatore-Torino.html']
['Quadrilocale via Bibiana, Torino', '€ 108.000', '2', '95', 'None', 'https://www.immobiliare.it/63854132-Vendita-Quadrilocale-via-Bibiana-Torino.html']
['Quadrilocale corso Monte Cucco, Torino', '€ 218.000', '4', '1', '130', 'https://www.immobiliare.it/63854506-Vendita-Quadrilocale-corso-Monte-Cucco-Torino.html']
['Bilocale via Erminio Macario 8, Chieri', '€ 90.000', '2', '2', '60', 'https://www.immobiliare.it/63857368-Vend

In [18]:
# now we need to substitute the print() with something that push that data on the db 
# how we can push some data there

# fists we create a session, that open a connection with the db
s = session()
# that we need to associate the data with an instance of our table class and post there
post = MyTable( address = 'the address i want to post',
                price = '10000',
                rooms = '2',
                bathrooms = '1',
                area = '100',
                url = 'http://myfakeurl.com/blablabla'
             )   
# add the post to the session
s.add(post)
# commit our post
s.commit()

In [19]:
# now we are ready to put all rows there
file = open('data.csv','r')
reader = csv.reader(file)
counter = 0 
for row in reader:
    if counter > 0:
        s = session()
        post = MyTable( address = row[0],
                    price =  row[1],
                    rooms =  row[2],
                    bathrooms =  row[3],
                    area =  row[4],
                    url =  row[5]
                 ) 
        s.add(post)
        s.commit()
        counter += 1
    else:
        counter += 1
        pass


In [20]:
# ok, and now what we can see that data?
# introducing Pandas
import pandas as pd
import sqlite3

# pandas can connect to the db and visualize it's content
conn = sqlite3.connect("database.sqlite")
df = pd.read_sql_query("SELECT * FROM mytable;", conn)
df


,id,address,price,rooms,bathrooms,area,url
0,1,the address i want to post,10000,2,1,100,http://myfakeurl.com/blablabla
1,2,"Trilocale via pinerolo, Rivalta di Torino",€ 78.000,3,1,None,https://www.immobiliare.it/63854720-Vendita-Tr...
2,3,"Bilocale via Luigi Salvatore Cherubini 20, Torino",€ 49.000,2,1,55,https://www.immobiliare.it/63858966-Vendita-Bi...
3,4,"Quadrilocale via Luigi Salvatore Cherubini 20,...",€ 79.000,4,1,85,https://www.immobiliare.it/63857262-Vendita-Qu...
4,5,"Quadrilocale via Bibiana, Torino",€ 108.000,2,95,None,https://www.immobiliare.it/63854132-Vendita-Qu...
5,6,"Quadrilocale corso Monte Cucco, Torino",€ 218.000,4,1,130,https://www.immobiliare.it/63854506-Vendita-Qu...
6,7,"Bilocale via Erminio Macario 8, Chieri",€ 90.000,2,2,60,https://www.immobiliare.it/63857368-Vendita-Bi...
7,8,"Appartamento via Amendola, Chieri",€ 260.000,5,2,155,https://www.immobiliare.it/63857376-Vendita-Ap...
8,9,"Trilocale via Modesto Moriondo 30, Rivalta di ...",€ 119.000,3,1,None,https://www.immobiliare.it/63852592-Vendita-Tr...
9,10,"Trilocale via Morazzone, Torino",99999999,3,1,75,https://www.immobiliare.it/63855156-Vendita-Tr...


# Pull out data from the SQLite DB

In [21]:
# pull out data from the db
s = session()
results = s.query(MyTable)
for r in results:
    print(r.id,r.address,r.price,r.rooms)
print('results: ',results.count())

1 the address i want to post 10000 2
2 Trilocale via pinerolo, Rivalta di Torino € 78.000 3
3 Bilocale via Luigi Salvatore Cherubini 20, Torino € 49.000 2
4 Quadrilocale via Luigi Salvatore Cherubini 20, Torino € 79.000 4
5 Quadrilocale via Bibiana, Torino € 108.000 2
6 Quadrilocale corso Monte Cucco, Torino € 218.000 4
7 Bilocale via Erminio Macario 8, Chieri € 90.000 2
8 Appartamento via Amendola, Chieri € 260.000 5
9 Trilocale via Modesto Moriondo 30, Rivalta di Torino € 119.000 3
10 Trilocale via Morazzone, Torino 99999999 3
11 Appartamento via Spano 14-6, Torino € 310.000 5+
12 Quadrilocale via Freinetto 51, Coazze € 92.000 4
13 Monolocale via Spano 14-6, Torino € 98.000 1
14 Appartamento strada Comunale Santa Margherita, Torino € 465.000 5
15 Trilocale via Riva 24, Moriondo Torinese € 155.000 3
16 Bilocale via Vittorio Emanuele II 47, Chieri € 125.000 2
17 Monolocale via Spano 14-6, Torino € 98.000 1
18 Bilocale via Piedicavallo, Torino € 58.000 2
19 Quadrilocale corso Roma, Monc

In [22]:
# pull out data from the db with filters 
s = session()
# shrink a little the work
results = s.query(MyTable).filter(MyTable.area != 'None')
for r in results:
    print(r.address,r.price,r.rooms)
print('results: ',results.count())

the address i want to post 10000 2
Bilocale via Luigi Salvatore Cherubini 20, Torino € 49.000 2
Quadrilocale via Luigi Salvatore Cherubini 20, Torino € 79.000 4
Quadrilocale corso Monte Cucco, Torino € 218.000 4
Bilocale via Erminio Macario 8, Chieri € 90.000 2
Appartamento via Amendola, Chieri € 260.000 5
Trilocale via Morazzone, Torino 99999999 3
Appartamento via Spano 14-6, Torino € 310.000 5+
Quadrilocale via Freinetto 51, Coazze € 92.000 4
Monolocale via Spano 14-6, Torino € 98.000 1
Appartamento strada Comunale Santa Margherita, Torino € 465.000 5
Trilocale via Riva 24, Moriondo Torinese € 155.000 3
Bilocale via Vittorio Emanuele II 47, Chieri € 125.000 2
Monolocale via Spano 14-6, Torino € 98.000 1
Bilocale via Piedicavallo, Torino € 58.000 2
Appartamento via Spano 14-6, Torino € 288.000 5+
Trilocale strada Comunale Superga 298, Torino € 220.000 3
Trilocale via Arona 7, Torino € 85.000 3
Quadrilocale via Spano 6-10, Torino € 288.000 4
Quadrilocale strada del Portone 35-6, Torino

In [23]:
# update a specific rows with ids 
s = session()
s.query(MyTable).filter(MyTable.id == 10).update({'price': '99999999'})
s.commit()

In [24]:
s = session()
# shrink a little the work
results = s.query(MyTable).filter(MyTable.id == 10)
for r in results:
    print(r.address,r.price,r.rooms)
print('results: ',results.count())

Trilocale via Morazzone, Torino 99999999 3
results:  1
